In [ ]:
# Download dataset from github repo
!rm -r sample_data
!git clone https://github.com/willjhliang/traffic-sign-recognition.git
!mv traffic-sign-recognition/data .
!rm -r traffic-sign-recognition

In [ ]:
import os
from copy import deepcopy
import itertools
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, KFold

In [ ]:
K = 58  # Number of classes
S = 32  # Size of image, dimension is (s, s, 3)

random_seed = 19104

# Data Exploration

In [ ]:
def load_data(datapath):
    data = {}
    for k in range(K):
        data[k] = []
    for f in os.listdir(datapath):
        k = int(f[:3])
        img = Image.open(os.path.join(datapath, f)).convert('L')
        img = np.asarray(img) / 255
        data[k].append(img)
    return data

In [ ]:
labels = pd.read_csv("data/labels.csv")

train_data = load_data('data/images/train')
test_data = load_data('data/images/test')

In [ ]:
plt.gray()
fig, axs = plt.subplots(6, 10)
fig.set_figheight(15)
fig.set_figwidth(15)
for k, (i, j) in itertools.zip_longest(range(K), list(itertools.product(range(6), range(10))), fillvalue=-1):
    axs[i,j].axis('off')
    if k >= 0:
        axs[i,j].imshow(train_data[k][0])


In [ ]:
img_dist = plt.bar(list(range(K)), [len(train_data[k]) for k in range(K)])

In [ ]:
import cv2

def center_crop(img, center_percentage):
    width, height = img.shape
    width_offset = int(width * (1 - center_percentage) / 2)
    height_offset = int(height * (1 - center_percentage) / 2)
    img = img[width_offset:width-width_offset, height_offset:height-height_offset]
    return img

def rotate_img(img, angle):
    height, width = img.shape
    center_x, center_y = (width // 2, height // 2)

    rot_mat = cv2.getRotationMatrix2D((center_x, center_y), angle, 1.0)
    cos = np.abs(rot_mat[0, 0])
    sin = np.abs(rot_mat[0, 1])

    new_width = int((height * sin) + (width * cos))
    new_height = int((height * cos) + (width * sin))
    rot_mat[0, 2] += (new_width / 2) - center_x
    rot_mat[1, 2] += (new_height / 2) - center_y

    img = cv2.warpAffine(img, rot_mat, (new_width, new_height))
    img = cv2.resize(img, (width, height))

    return img

def shift_brightness(img, shift):
    img = np.clip(img + shift, 0, 255)
    return img


In [ ]:
from random import randint

def augment_img(img):
    rot_angle = randint(-30, 30)
    crop_center_percentage = randint(70, 90) / 100
    crop_center_percentage = 0.8
    brightness_shift = randint(-20, 20) / 100
    img = rotate_img(img, rot_angle)
    img = center_crop(img, crop_center_percentage)
    img = shift_brightness(img, brightness_shift)
    img = center_crop(img, 0.8)
    return img

In [ ]:
largest_class_size = max([len(train_data[k]) for k in range(K)])
for k in range(K):
    size_diff = largest_class_size - len(train_data[k])
    for i in range(size_diff):
        train_data[k].append(augment_img(train_data[k][i % len(train_data[k])]))

In [ ]:
img_dist = plt.bar(list(range(K)), [len(train_data[k]) for k in range(K)])

In [ ]:
def prepare_data(data):
    X = []
    y = []
    for k in range(K):
        for i in data[k]:
            X.append(cv2.resize(i, (S, S)))
            y.append(k)
    
    X_flattened = deepcopy(X)
    for i in range(len(X_flattened)):
        X_flattened[i] = X_flattened[i].flatten()
    
    X = np.array(X)
    X_flattened = np.array(X_flattened)
    y = np.array(y)
    return X, X_flattened, y

In [ ]:
X_train, X_train_flattened, y_train = prepare_data(train_data)
X_test, X_test_flattened, y_test = prepare_data(test_data)

# Models

The following is a set of models we run on the data. Starting with the most simple baseline K-Nearest Neighbors, we move toward more complex models.
1. Baseline KNN
2. Adaboost
3. Logistic Regression
4. Kernelized SVM
5. Dense Neural Network
6. Convolutional Neural Network

We also test two more advanced strategies.
1. Autoencoder dimensionality reduction allows us to embed the images in a lower dimensional space, which may lead to stronger classification performance by simpler models.
2. Transfer learning with a CNN allows us to adapt weights from pre-trained networks to our traffic sign recognition problem.

## Baseline KNN

Train a baseline K-Nearest Neighbors models to classify traffic sign images. Use 10-Fold cross validation to determine the best value of K

In [ ]:
def knnBaseline(X_train, y_train, X_test, y_test):
    kf = KFold(n_splits = 10)
    best_k = -1
    best_acc = 0
    val_accuracies = []

    # Iterate through possible values of k from 1 to 30, incrementing by 2
    for k_neighbors in range(1, 30, 2):
        for train_index, val_index in kf.split(X_train): # Iterate through all 10 folds
            total_acc = 0
            # Split data into training data and validation data
            X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
            y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

            # Train KNN model
            knn_model = KNeighborsClassifier(n_neighbors=k_neighbors)
            knn_model.fit(X_train_fold, y_train_fold)
            total_acc = total_acc + knn_model.score(X_val_fold, y_val_fold)
        
        # Get avg accuracy for the folds for this k value
        avg_acc = total_acc / 10
        val_accuracies.append(avg_acc)
        if avg_acc > best_acc:
            best_acc = avg_acc
            best_k = k_neighbors

    # Plot to show the best values
    plt.plot(list(range(1, 30, 2)), val_accuracies)
    plt.show()
    print("Best k: ", best_k)

    # Fit model with the best k value
    model = KNeighborsClassifier(n_neighbors=best_k)
    model.fit(X_train_flattened, y_train)
    
    # Accurcay on the test set
    return model.score(X_test_flattened, y_test)


In [ ]:
print(knnBaseline(X_train_flattened, y_train, X_test_flattened, y_test))

## Adaboost

## Logistic Regression

## Kernelized SVM

## Dense Neural Network

## Convolutional Neural Network

## Autoencoder Dimensionality Reduction

### Adaboost

### Logistic Regression

### Kernelized SVM

## Transfer Learning with CNN